# 第6章 使用一等函数实现设计模式

## 策略模式

In [5]:
# 上下文：把计算委托给实现不同算法的可互换组件，它提供服务。上下文是Order，它会根据不同的算法计算促销折扣
# 策略：实现不同算法的组件共同的接口
#       策略1：有 1000 或以上积分的顾客，每个订单享 5% 折扣
#       策略2：同一订单中，单个商品的数量达到 20 个或以上，享 10% 折扣
#       策略3：订单中的不同商品达到 10 个或以上，享 7% 折扣
# 具体策略：策略的具体子类。

from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple("Customer", "name fidelity")  # 通过具名元组定义一个”顾客“类型，包含【顾客名称、顾客积分】

In [5]:
class LineItem:
    """定义清单明细"""

    def __init__(self, product, quantity, price):
        """
        定义单个物品的购买属性
        """
        self.product = product  # 物品
        self.quantity = quantity  # 数量
        self.price = price  # 单价

    def total(self):
        """明细价格：价格*数量"""
        return self.price * self.quantity

In [5]:
class Order:
    """订单信息"""

    def __init__(self, customer, cart, promotion):
        
        self.customer = customer  # 顾客
        self.cart = list(cart)  # 购物车
        self.promotion = promotion  # 折扣策略

    def total(self):
        """总价格（未打折）= 购物车中明细价格之和"""
        
        if not hasattr(self, '__total'):
            return sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        """最终价格/确定价格 总价格-折扣价格"""
        
        if self.promotion is None:
            discount = 0
        else:
            # 传入一个当前对象
            discount = self.promotion.discount(self)
        return self.total() - discount

    def __repr__(self):
        
        fmt = '<Order total:{:.2f} due:{:.2f}>'
        return fmt.format(self.total(), self.due())

In [5]:
class Promotion(ABC):
    """策略，抽象基类"""

    @abstractmethod
    def discount(self, order):
        """返回折扣金额（正值）"""


class FidelityPromo(Promotion):
    
    def discount(self, order):

        # 策略1：有 1000 或以上积分的顾客，每个订单享 5% 折扣
        return order.total() * .05 if order.customer.fidelity > 1000 else 0


class BulkItemPromo(Promotion):
    
    def discount(self, order):
        
        # 策略2：同一订单中，单个商品的数量达到 20 个或以上，享 10% 折扣
        discout = 0
        for item in order.cart:
            if item.quantity >= 20:
                discout += item.total() * .1
                
        return discout


class LargeOrderPromo(Promotion):
    
    def discount(self, order):
        
        # 策略3：订单中的不同商品达到 10 个或以上，享 7% 折扣
        # return order.total() * .07 if len(order.cart) >= 10 else 0
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * .07
        
        return 0

In [12]:
# 两个顾客：joe 的积分是 0，ann 的积分是 1100。
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
# 物品、数量和单价
# 2 + 15 + 25
cart = [LineItem('banana', 4, .5), LineItem('apple', 10, 1.5), LineItem('watermellon', 5, 5.0)]
print(Order(joe, cart, FidelityPromo())) # <Order total:42.00 due:42.00> 不打折

<Order total:42.00 due:42.00>


In [9]:
print(Order(ann, cart, FidelityPromo())) # <Order total:42.00 due:39.90> 策略1：有 1000 或以上积分的顾客，每个订单享 5% 折扣

<Order total:42.00 due:39.90>


In [10]:
banana_cart = [LineItem('banana', 30, .5), LineItem('apple', 10, 1.5)]
print(Order(joe, banana_cart, BulkItemPromo())) #<Order total:30.00 due:28.50> # 策略2：同一订单中，单个商品的数量达到 20 个或以上，享 10% 折扣

<Order total:30.00 due:28.50>


In [11]:
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
print(Order(joe, long_order, LargeOrderPromo())) #<Order total:10.00 due:9.30> 策略3：订单中的不同商品达到 10 个或以上，享 7% 折扣
print(Order(joe, cart, LargeOrderPromo()))

<Order total:10.00 due:9.30>
<Order total:42.00 due:42.00>


## 使用函数实现"策略"模型

In [15]:
# 优化6-2 没必要在新建订单时实例化新的促销对象，函数拿来即用
# 上下文：把计算委托给实现不同算法的可互换组件，它提供服务。上下文是Order，它会根据不同的算法计算促销折扣
# 策略：实现不同算法的组件共同的接口
#       策略1：有 1000 或以上积分的顾客，每个订单享 5% 折扣
#       策略2：同一订单中，单个商品的数量达到 20 个或以上，享 10% 折扣
#       策略3：订单中的不同商品达到 10 个或以上，享 7% 折扣
# 具体策略：策略的具体子类。
from collections import namedtuple

Customer = namedtuple("Customer", "name fidelity")  # 通过具名元组定义一个”顾客“类型，包含【顾客名称、顾客积分】

In [15]:
class LineItem:
    """定义清单明细"""

    def __init__(self, product, quantity, price):
        self.product = product  # 物品
        self.quantity = quantity  # 数量
        self.price = price  # 单价

    def total(self):
        """明细价格：价格*数量"""
        return self.price * self.quantity


class Order:
    """订单信息"""

    def __init__(self, customer, cart, promotion):
        self.customer = customer  # 顾客
        self.cart = list(cart)  # 购物车
        self.promotion = promotion  # 折扣策略

    def total(self):
        """总价格（未打折）= 购物车中明细价格之和"""
        if not hasattr(self, '__total'):
            return sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        """最终价格/确定价格 总价格-折扣价格"""
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total:{:.2f} due:{:.2f}>'
        return fmt.format(self.total(), self.due())

def fidelity_promo(order):
    # 策略1：有 1000 或以上积分的顾客，每个订单享 5% 折扣
    return order.total() * .05 if order.customer.fidelity > 1000 else 0

def bulk_item_promo(order):
    # 策略2：同一订单中，单个商品的数量达到 20 个或以上，享 10% 折扣
    discout = 0
    for item in order.cart:
        if item.quantity >= 20:
            discout += item.total() * .1
    return discout

def large_order_promo(order):
    # 策略3：订单中的不同商品达到 10 个或以上，享 7% 折扣
    # return order.total() * .07 if len(order.cart) >= 10 else 0
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

In [26]:
# 两个顾客：joe 的积分是 0，ann 的积分是 1100。

joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5), LineItem('apple', 10, 1.5), LineItem('watermellon', 5, 5.0)]
print(Order(joe, cart, fidelity_promo))  # <Order total:42.00 due:42.00> 不打折
print(Order(ann, cart, fidelity_promo))  # <Order total:42.00 due:39.90> 策略1：有 1000 或以上积分的顾客，每个订单享 5% 折扣
banana_cart = [LineItem('banana', 30, .5), LineItem('apple', 10, 1.5)]
print(Order(joe, banana_cart, bulk_item_promo))  # <Order total:30.00 due:28.50> # 策略2：同一订单中，单个商品的数量达到 20 个或以上，享 10% 折扣
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
print(Order(joe, long_order, large_order_promo))  # <Order total:10.00 due:9.30> 策略3：订单中的不同商品达到 10 个或以上，享 7% 折扣
print(Order(joe, cart, large_order_promo))

<Order total:42.00 due:42.00>
<Order total:42.00 due:39.90>
<Order total:30.00 due:28.50>
<Order total:10.00 due:9.30>
<Order total:42.00 due:42.00>


## 选择最佳策略：简单的方式

In [27]:
# 找到最优的优惠方案
# 上下文：把计算委托给实现不同算法的可互换组件，它提供服务。上下文是Order，它会根据不同的算法计算促销折扣
# 策略：实现不同算法的组件共同的接口
#       策略1：有 1000 或以上积分的顾客，每个订单享 5% 折扣
#       策略2：同一订单中，单个商品的数量达到 20 个或以上，享 10% 折扣
#       策略3：订单中的不同商品达到 10 个或以上，享 7% 折扣
# 具体策略：策略的具体子类。

# from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple("Customer", "name fidelity")  # 通过具名元组定义一个”顾客“类型，包含【顾客名称、顾客积分】


class LineItem:
    """定义清单明细"""

    def __init__(self, product, quantity, price):
        self.product = product  # 物品
        self.quantity = quantity  # 数量
        self.price = price  # 单价

    def total(self):
        """明细价格：价格*数量"""
        return self.price * self.quantity


class Order:
    """订单信息"""

    def __init__(self, customer, cart, promotion):
        self.customer = customer  # 顾客
        self.cart = list(cart)  # 购物车
        self.promotion = promotion  # 折扣策略

    def total(self):
        """总价格（未打折）= 购物车中明细价格之和"""
        if not hasattr(self, '__total'):
            return sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        """最终价格/确定价格 总价格-折扣价格"""
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total:{:.2f} due:{:.2f}>'
        return fmt.format(self.total(), self.due())


def fidelity_promo(order):
    # 策略1：有 1000 或以上积分的顾客，每个订单享 5% 折扣
    return order.total() * .05 if order.customer.fidelity > 1000 else 0


def bulk_item_promo(order):
    # 策略2：同一订单中，单个商品的数量达到 20 个或以上，享 10% 折扣
    discout = 0
    for item in order.cart:
        if item.quantity >= 20:
            discout += item.total() * .1
    return discout


def large_order_promo(order):
    # 策略3：订单中的不同商品达到 10 个或以上，享 7% 折扣
    # return order.total() * .07 if len(order.cart) >= 10 else 0
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0


def best_promo(order):
    """
    选择最优的折扣方案
    :param order: 订单
    :return: 最优解
    """
    promos = [fidelity_promo, bulk_item_promo, large_order_promo]  # 列出函数实现的各个策略
    return max(promo(order) for promo in promos)

In [28]:
# 两个顾客：joe 的积分是 0，ann 的积分是 1100。
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
# 三种购物车
cart = [LineItem('banana', 4, .5), LineItem('apple', 10, 1.5), LineItem('watermellon', 5, 5.0)]
banana_cart = [LineItem('banana', 30, .5), LineItem('apple', 10, 1.5)]
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]

In [29]:
print("------------joe的选择（普通购物车）------------")
print(Order(joe, cart, fidelity_promo))  # <Order total:42.00 due:42.00> 不打折
print(Order(joe, cart, bulk_item_promo))  # <Order total:30.00 due:28.50> # 策略2：同一订单中，单个商品的数量达到 20 个或以上，享 10% 折扣
print(Order(joe, cart, large_order_promo))  # <Order total:10.00 due:9.30> 策略3：订单中的不同商品达到 10 个或以上，享 7% 折扣
print(Order(joe, cart, best_promo))  # <Order total:10.00 due:9.30> 策略3：订单中的不同商品达到 10 个或以上，享 7% 折扣

------------joe的选择（普通购物车）------------
<Order total:42.00 due:42.00>
<Order total:42.00 due:42.00>
<Order total:42.00 due:42.00>
<Order total:42.00 due:42.00>


In [30]:
print("------------ann的选择（普通购物车）------------")
print(Order(ann, cart, fidelity_promo))  # <Order total:42.00 due:42.00> 不打折
print(Order(ann, cart, bulk_item_promo))  # <Order total:30.00 due:28.50> # 策略2：同一订单中，单个商品的数量达到 20 个或以上，享 10% 折扣
print(Order(ann, cart, large_order_promo))  # <Order total:10.00 due:9.30> 策略3：订单中的不同商品达到 10 个或以上，享 7% 折扣
print(Order(ann, cart, best_promo))  # <Order total:10.00 due:9.30> 策略3：订单中的不同商品达到 10 个或以上，享 7% 折扣

------------ann的选择（普通购物车）------------
<Order total:42.00 due:39.90>
<Order total:42.00 due:42.00>
<Order total:42.00 due:42.00>
<Order total:42.00 due:39.90>


In [31]:
print("------------joe的选择（香蕉购物车）------------")
print(Order(joe, banana_cart, fidelity_promo))  # <Order total:42.00 due:42.00> 不打折
print(Order(joe, banana_cart, bulk_item_promo))  # <Order total:30.00 due:28.50> # 策略2：同一订单中，单个商品的数量达到 20 个或以上，享 10% 折扣
print(Order(joe, banana_cart, large_order_promo))  # <Order total:10.00 due:9.30> 策略3：订单中的不同商品达到 10 个或以上，享 7% 折扣
print(Order(joe, banana_cart, best_promo))  # <Order total:10.00 due:9.30> 策略3：订单中的不同商品达到 10 个或以上，享 7% 折扣

------------joe的选择（香蕉购物车）------------
<Order total:30.00 due:30.00>
<Order total:30.00 due:28.50>
<Order total:30.00 due:30.00>
<Order total:30.00 due:28.50>


In [32]:
print("------------joe的选择（长订单购物车）------------")
print(Order(joe, long_order, fidelity_promo))  # <Order total:42.00 due:42.00> 不打折
print(Order(joe, long_order, bulk_item_promo))  # <Order total:30.00 due:28.50> # 策略2：同一订单中，单个商品的数量达到 20 个或以上，享 10% 折扣
print(Order(joe, long_order, large_order_promo))  # <Order total:10.00 due:9.30> 策略3：订单中的不同商品达到 10 个或以上，享 7% 折扣
print(Order(joe, long_order, best_promo))  # <Order total:10.00 due:9.30> 策略3：订单中的不同商品达到 10 个或以上，享 7% 折扣

------------joe的选择（长订单购物车）------------
<Order total:10.00 due:10.00>
<Order total:10.00 due:10.00>
<Order total:10.00 due:9.30>
<Order total:10.00 due:9.30>


## 选择最佳策略：灵活的方式

In [33]:
#  6-6 的有一个缺陷：若添加新的促销策略，要定义相应的函数，需要及得把它添加到 promos 列表中。否则 best_promo 不会考虑它
#  globals() 返回一个字典，表示当前的全局符号表。 使用 globals 函数帮助 best_promo 自动找到其他可用的 *_promo 函数，过程有点 曲折。

# from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple("Customer", "name fidelity")  # 通过具名元组定义一个”顾客“类型，包含【顾客名称、顾客积分】


class LineItem:
    """定义清单明细"""

    def __init__(self, product, quantity, price):
        self.product = product  # 物品
        self.quantity = quantity  # 数量
        self.price = price  # 单价

    def total(self):
        """明细价格：价格*数量"""
        return self.price * self.quantity


class Order:
    """订单信息"""

    def __init__(self, customer, cart, promotion):
        self.customer = customer  # 顾客
        self.cart = list(cart)  # 购物车
        self.promotion = promotion  # 折扣策略

    def total(self):
        """总价格（未打折）= 购物车中明细价格之和"""
        if not hasattr(self, '__total'):
            return sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        """最终价格/确定价格 总价格-折扣价格"""
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total:{:.2f} due:{:.2f}>'
        return fmt.format(self.total(), self.due())


def fidelity_promo(order):
    # 策略1：有 1000 或以上积分的顾客，每个订单享 5% 折扣
    return order.total() * .05 if order.customer.fidelity > 1000 else 0


def bulk_item_promo(order):
    # 策略2：同一订单中，单个商品的数量达到 20 个或以上，享 10% 折扣
    discout = 0
    for item in order.cart:
        if item.quantity >= 20:
            discout += item.total() * .1
    return discout


def large_order_promo(order):
    # 策略3：订单中的不同商品达到 10 个或以上，享 7% 折扣
    # return order.total() * .07 if len(order.cart) >= 10 else 0
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0


def best_promo(order):
    """
    选择最优的折扣方案
    :param order: 订单
    :return: 最优解
    """
    # promos = [fidelity_promo, bulk_item_promo, large_order_promo]  # 列出函数实现的各个策略
    # 直接在全局进行结果的搜索
    promos = [globals()[name] for  name in globals() if name.endswith('_promo') and name != 'best_promo']
    
    return max(promo(order) for promo in promos)

In [34]:
# 两个顾客：joe 的积分是 0，ann 的积分是 1100。
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
# 三种购物车
cart = [LineItem('banana', 4, .5), LineItem('apple', 10, 1.5), LineItem('watermellon', 5, 5.0)]
banana_cart = [LineItem('banana', 30, .5), LineItem('apple', 10, 1.5)]
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]

print("------------joe的选择（普通购物车）------------")
print(Order(joe, cart, fidelity_promo))  # <Order total:42.00 due:42.00> 不打折
print(Order(joe, cart, bulk_item_promo))  # <Order total:30.00 due:28.50> # 策略2：同一订单中，单个商品的数量达到 20 个或以上，享 10% 折扣
print(Order(joe, cart, large_order_promo))  # <Order total:10.00 due:9.30> 策略3：订单中的不同商品达到 10 个或以上，享 7% 折扣
print(Order(joe, cart, best_promo))  # <Order total:10.00 due:9.30> 策略3：订单中的不同商品达到 10 个或以上，享 7% 折扣
print("------------ann的选择（普通购物车）------------")
print(Order(ann, cart, fidelity_promo))  # <Order total:42.00 due:42.00> 不打折
print(Order(ann, cart, bulk_item_promo))  # <Order total:30.00 due:28.50> # 策略2：同一订单中，单个商品的数量达到 20 个或以上，享 10% 折扣
print(Order(ann, cart, large_order_promo))  # <Order total:10.00 due:9.30> 策略3：订单中的不同商品达到 10 个或以上，享 7% 折扣
print(Order(ann, cart, best_promo))  # <Order total:10.00 due:9.30> 策略3：订单中的不同商品达到 10 个或以上，享 7% 折扣
print("------------joe的选择（香蕉购物车）------------")
print(Order(joe, banana_cart, fidelity_promo))  # <Order total:42.00 due:42.00> 不打折
print(Order(joe, banana_cart, bulk_item_promo))  # <Order total:30.00 due:28.50> # 策略2：同一订单中，单个商品的数量达到 20 个或以上，享 10% 折扣
print(Order(joe, banana_cart, large_order_promo))  # <Order total:10.00 due:9.30> 策略3：订单中的不同商品达到 10 个或以上，享 7% 折扣
print(Order(joe, banana_cart, best_promo))  # <Order total:10.00 due:9.30> 策略3：订单中的不同商品达到 10 个或以上，享 7% 折扣
print("------------joe的选择（长订单购物车）------------")
print(Order(joe, long_order, fidelity_promo))  # <Order total:42.00 due:42.00> 不打折
print(Order(joe, long_order, bulk_item_promo))  # <Order total:30.00 due:28.50> # 策略2：同一订单中，单个商品的数量达到 20 个或以上，享 10% 折扣
print(Order(joe, long_order, large_order_promo))  # <Order total:10.00 due:9.30> 策略3：订单中的不同商品达到 10 个或以上，享 7% 折扣
print(Order(joe, long_order, best_promo))  # <Order total:10.00 due:9.30> 策略3：订单中的不同商品达到 10 个或以上，享 7% 折扣

------------joe的选择（普通购物车）------------
<Order total:42.00 due:42.00>
<Order total:42.00 due:42.00>
<Order total:42.00 due:42.00>
<Order total:42.00 due:42.00>
------------ann的选择（普通购物车）------------
<Order total:42.00 due:39.90>
<Order total:42.00 due:42.00>
<Order total:42.00 due:42.00>
<Order total:42.00 due:39.90>
------------joe的选择（香蕉购物车）------------
<Order total:30.00 due:30.00>
<Order total:30.00 due:28.50>
<Order total:30.00 due:30.00>
<Order total:30.00 due:28.50>
------------joe的选择（长订单购物车）------------
<Order total:10.00 due:10.00>
<Order total:10.00 due:10.00>
<Order total:10.00 due:9.30>
<Order total:10.00 due:9.30>


In [41]:
for name in globals():
    if name.endswith("_promo"):
        print(name)

fidelity_promo
bulk_item_promo
large_order_promo
best_promo
